In [40]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
with torch.profiler.profile() as profiler:
        pass
from sklearn.preprocessing import MinMaxScaler
import math

from operator import itemgetter
import operator

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
def build_network(f1_layer_size, f2_layer_size, dropout, num_layers=2):


    network = nn.Sequential(  # fully-connected, dual hidden layer
        nn.Linear(128, f1_layer_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(f1_layer_size, f2_layer_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(f2_layer_size, 8),
        nn.Softmax(dim=1),
    )

    return network


In [4]:
torch.set_default_tensor_type(torch.FloatTensor)

In [5]:
torch.__version__

'1.12.1'

In [6]:
class DogInput:
    def __init__(self, dogid, raceid,stats, dog, lstmCellh,lstmCellc) -> None:
        self.dogid= dogid
        self.raceid = raceid
        self.stats = stats.to('cuda:0')
        self.dog = dog
        self.lstmCellh = lstmCellh.float().to('cuda:0')
        self.lstmCellc = lstmCellc.float().to('cuda:0')
        self.visited = 0
        
        
    def lstm_i(self, lstmInput):
        (self.lstmCellh,self.lstmCellc) = lstmInput
        # self.lstmCellh=self.lstmCellh.to(device)
        # self.lstmCellc=self.lstmCellc.to(device)
        self.visited = self.visited + 1
        # if self.visited>1:
        #     print("FOUND LEAK")
        #     sasdfasd

    def nextrace(self, raceid):
        self.nextrace_id = raceid

    def prevrace(self, raceid):
        self.prevrace_id = raceid

    def lstm_o(self, lstm_o):
        # print(lstm_o[0]._version)
        (lh,lc) = lstm_o
        if self.nextrace_id==-1:
            pass
        else:
            self.dog.races[self.nextrace_id].lstm_i((lh.detach(), lc.detach()))
            


In [7]:
class Dog:
    def __init__(self, dogid, hidden_size, layers) -> None:
        self.dogid = dogid
        # self.raceids = raceids #possible dictionary of race id keys dog stat outs
        self.lstmcell = 0
        self.layers = layers
        self.hidden_size = hidden_size
        self.l_debug = None
        self.races = {}

    def add_races(self, raceid, racedate, stats,nextraceid, prevraceid):
        self.races[raceid] = DogInput(self.dogid, raceid, stats, self, torch.randn(self.hidden_size).clone(),torch.randn(self.hidden_size).clone()) #this is the change
        self.races[raceid].nextrace(nextraceid)
        self.races[raceid].prevrace(prevraceid)

In [19]:
class Race:
    def __init__(self, raceid,trackOHE, dist, classes):
        self.raceid = raceid
        self.race_dist = dist.to('cuda:0')
        self.race_track = trackOHE.to('cuda:0')
        self.classes =  classes.to('cuda:0')

    def add_dogs(self, dogs_list:DogInput):
        self.dog1 = dogs_list[0]
        self.dog2 = dogs_list[1]
        self.dog3 = dogs_list[2]
        self.dog4 = dogs_list[3]
        self.dog5 = dogs_list[4]
        self.dog6 = dogs_list[5]
        self.dog7 = dogs_list[6]
        self.dog8 = dogs_list[7]
        self.dogs = dogs_list

    def nn_input(self):
        input = torch.cat([x.stats for x in self.dogs], dim = 0)
        full_input = torch.cat((self.race_dist,self.race_track, input), dim=0).to(device='cuda:0')
        self.full_input = full_input
        return full_input

    def lstm_input(self):
        
        l_input = [x.lstmCell for x in self.dogs]
        return l_input

    def list_dogs(self):
        dogs_l = [x for x in self.dogs]
        return dogs_l

    def pass_lstm_output(self, lstms):
        for i,dog in enumerate(self.dogs):
            
            lh,lc = lstms[i]
            #print(f"{dog.dogid,lh,lc=}")
            lh,lc = lh.detach(), lc.clone()
            dog.lstm_o((lh,lc))
            dog.dog.l_debug = (lh,lc)
        # zipped_lstm = zip(self.dogs,lstms)
        # [x.lstm_o(y) for x,y in zipped_lstm]

In [32]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()
        self.lstm1 = nn.LSTMCell(input_size, hidden_size)
        self.lstm2 = nn.LSTMCell(input_size, hidden_size)
        self.lstm3 = nn.LSTMCell(input_size, hidden_size)
        self.lstm4 = nn.LSTMCell(input_size, hidden_size)
        self.lstm5 = nn.LSTMCell(input_size, hidden_size)
        self.lstm6 = nn.LSTMCell(input_size, hidden_size)
        self.lstm7 = nn.LSTMCell(input_size, hidden_size)
        self.lstm8 = nn.LSTMCell(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size * 8, 64)
        self.fc3 = nn.Linear(64, 8)
        self.hidden_size = hidden_size

    # x represents our data
    def forward(self, race: Race):
        #x = race.nn_input().float().to('cuda:0')
        x = race.full_input.float()

        (h1, c1) = self.lstm1(x, (race.dog1.lstmCellh, race.dog1.lstmCellc))
        (h2, c2) = self.lstm2(x, (race.dog2.lstmCellh, race.dog2.lstmCellc))
        (h3, c3) = self.lstm3(x, (race.dog3.lstmCellh, race.dog3.lstmCellc))
        (h4, c4) = self.lstm4(x, (race.dog4.lstmCellh, race.dog4.lstmCellc))
        (h5, c5) = self.lstm5(x, (race.dog5.lstmCellh, race.dog5.lstmCellc))
        (h6, c6) = self.lstm6(x, (race.dog6.lstmCellh, race.dog6.lstmCellc))
        (h7, c7) = self.lstm7(x, (race.dog7.lstmCellh, race.dog7.lstmCellc))
        (h8, c8) = self.lstm8(x, (race.dog8.lstmCellh, race.dog8.lstmCellc))

        lstm_list = [
            (h1, c1),
            (h2, c2),
            (h3, c3),
            (h4, c4),
            (h5, c5),
            (h6, c6),
            (h7, c7),
            (h8, c8)
        ]

        race.pass_lstm_output(lstm_list)

        xhh = torch.cat((h1.clone(),h2.clone(), h3.clone(), h4.clone(), h5.clone(), h6.clone(), h7.clone(), h8.clone()), dim=0).clone()
        xh = self.fc2(xhh)
        xf = self.fc3(xh)

        output = F.softmax(xf, dim=0)
        return output


In [10]:
class Races:
    def __init__(self, hidden_size, layers, batch_size = 100) -> None:
        self.racesDict = {}
        self.dogsDict = {}
        self.raceIDs = []
        self.hidden_size = hidden_size
        self.layers = layers
        self.getter = operator.itemgetter(*range(batch_size))

    def add_race(self,raceid:str, trackOHE, dist, classes):
        self.racesDict[raceid] = Race(raceid, trackOHE, dist, classes)
        self.raceIDs.append(raceid)

    def add_dog(self,dogid):
        if dogid not in self.dogsDict.keys():
            self.dogsDict[dogid] = Dog(dogid, self.hidden_size, self.layers)
        else:
            self.dogsDict[dogid] = self.dogsDict[dogid]

    def get_race_input(self, idx) -> Race:
        
        raceidx = self.raceIDs[idx]
        # input = torch.cat([x.stats for x in self.racesDict[raceidx].dogs.values()], dim = 0)
        # full_input = torch.cat((self.racesDict[raceidx].race_dist,self.racesDict[raceidx].race_track, input), dim=0 )
        # dogs = [x for x in self.racesDict[raceidx].dogs]
        
        return self.racesDict[raceidx]

    def get_race_classes(self, idx):
        raceidx = self.raceIDs[idx]
        classes = [x for x in self.raceDict[raceidx].classes]
        return classes


In [46]:
f = operator.itemgetter(*range(1,100))

In [47]:
f(raceDB.racesDict)

KeyError: 1

In [11]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df.npy', 'rb')
resultsdf = pickle.load(dog_stats_file)
dog_stats_df = pd.DataFrame(resultsdf)
dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
# dog_stats_df.to_excel('dog_stats_df.xlsx')
# dog_stats_df = dog_stats_df[dog_stats_df['dogid']=='674060006']
# dog_stats_df = dog_stats_df[dog_stats_df['raceid']!="130628253"]

In [12]:
print(dog_stats_df.size)
print(dog_stats_df.drop_duplicates(subset=['dogid', 'raceid']).size)

5270640
5270640


In [13]:
dog_stats_df[dog_stats_df['dogid']=="-159632"]

,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP
2524,-159632,132951399,2016-07-11,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",450.0,"[6, 0.059524999999999995, -1, 0.05952499999999...",7.0,17.0,9.00,25.000000
8743,-159632,139325114,2016-08-02,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",400.0,"[8, 0.05930694444444444, 0.0003083771240174653...",6.0,11.2,6.70,18.766242
20697,-159632,151780470,2016-09-13,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",460.0,"[1, 0.05887962962962962, 0.0007715841355220415...",5.0,19.3,5.96,16.990319
44437,-159632,177083976,2016-12-05,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",450.0,"[3, 0.058605374396135265, 0.000835319370773274...",8.0,19.0,10.10,37.180216
71516,-159632,1967920009,2016-01-26,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",400.0,"[8, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",5.0,3.8,14.33,-1.000000


In [14]:
dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)

In [15]:
x = [1,2,3,4]
torch.tensor(x)

tensor([1, 2, 3, 4])

In [17]:
dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"])
layers = 1
hidden_size = 5
raceDB = Races(hidden_size,1)
print(raceDB)
for i,j in tqdm(dog_stats_group):
    #print(f"dog id = {i}\nlist = {j}")
    #tqdm.write(f"{i}")

    #creates next race column
    
    #j = j.sort_values(['date'])
    j["next_race"] = j["raceid"].shift(-1).fillna(-1)
    j["prev_race"] = j["raceid"].shift(1).fillna(-1)
    raceDB.add_dog(i)

    
    j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),x['next_race'], x['prev_race']), axis=1)
    



100%|██████████| 25187/25187 [03:49<00:00, 109.81it/s]


In [21]:
softmin = nn.Softmin()

races_group = dog_stats_df.groupby(['raceid'])
null_dog = Dog("nullDog", raceDB.hidden_size, raceDB.layers)
null_dog_i = DogInput("nullDog", "-1", torch.zeros(16), null_dog, torch.zeros(raceDB.hidden_size), torch.zeros(raceDB.hidden_size))
null_dog_i.nextrace(-1)
null_dog_i.prevrace(-1)
for i,j in tqdm(races_group):
   dist = torch.tensor([j.dist.iloc[0]])
   trackOHE = torch.tensor(j.trackOHE.iloc[0])
   margin_list = [x for x in j["place"]]
   adjustedMargin = margin_list + ([8] * (8 - len(margin_list)))
   adjustedMargin = softmin(torch.tensor(adjustedMargin)).to('cuda:0')
   raceDB.add_race(i,trackOHE,dist, adjustedMargin)
   
   
   dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]] # List of Dog Input??
   adjustedList = dog_list + ([null_dog_i] * (8 - len(dog_list)))
   raceDB.racesDict[i].add_dogs(adjustedList)
   raceDB.racesDict[i].nn_input()
 # need to add in
    #j.apply(lambda x: raceDB.racesDict[i].add_dog(x['dogid'], raceDB.dogsDict[x['dogid']].races[i]), axis=1)

  0%|          | 0/72073 [00:00<?, ?it/s]C:\Users\Nick\AppData\Local\Temp\ipykernel_17416\1246265129.py:13: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  adjustedMargin = softmin(torch.tensor(adjustedMargin)).to('cuda:0')
100%|██████████| 72073/72073 [00:54<00:00, 1326.18it/s]


In [39]:
raceDB.get_race_input([1])

TypeError: list indices must be integers or slices, not list

In [33]:
newnet = Net(144,5)
#x = newnet.forward(raceDB.racesDict["132951399"])
#print(x)

In [34]:
loss_function = nn.MSELoss()
optimizer = optim.Adam(newnet.parameters(), lr=0.001)

In [35]:
newnet = newnet.to('cuda:0')
newnet.forward

<bound method Net.forward of Net(
  (lstm1): LSTMCell(144, 5)
  (lstm2): LSTMCell(144, 5)
  (lstm3): LSTMCell(144, 5)
  (lstm4): LSTMCell(144, 5)
  (lstm5): LSTMCell(144, 5)
  (lstm6): LSTMCell(144, 5)
  (lstm7): LSTMCell(144, 5)
  (lstm8): LSTMCell(144, 5)
  (fc2): Linear(in_features=40, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)>

In [36]:
torch.autograd.set_detect_anomaly(True)
list_t = [] 
for epoch in range(3): 
    for i in trange(70000):  
        race = raceDB.get_race_input(i)
        X = race
        y = race.classes.to(device)
        #print(y)
        output = newnet(X)
        #
        #print(output) 
        #print([x.dogid for x in X.dogs])
        loss = loss_function(output, y)
        optimizer.zero_grad()
        #newnet.zero_grad()
        loss.backward(retain_graph=True)  
        optimizer.step()
        if i %1000 == 0:
            print(loss)
    # optimizer.step() 
    print(loss)

  0%|          | 3/70000 [00:02<10:59:18,  1.77it/s]

tensor(0.0412, device='cuda:0', grad_fn=<MseLossBackward0>)


  1%|▏         | 1003/70000 [01:11<1:18:44, 14.61it/s]

tensor(0.0408, device='cuda:0', grad_fn=<MseLossBackward0>)


  2%|▏         | 1052/70000 [01:14<1:21:29, 14.10it/s]


KeyboardInterrupt: 